In [ ]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler
# from sklearn.externals import joblib
import numpy as np

In [ ]:
# Load data
df_original = pd.read_excel("creep_data.xlsx")
original_data = df_original.values[:, 1:-1]

In [ ]:
# Get original data min and max values
original_min = np.min(original_data, axis=0)
original_max = np.max(original_data, axis=0)

In [ ]:
# Calculate cluster centers
cluster_centers = []
for index in range(8):
    cluster_data = pd.read_excel("data_files/multi-factors_clusters_912.xls", sheetname="cluster_" + str(index))
    cluster_dataX = cluster_data.values[:, 1:28]
    normalize_data = [(row - original_min) / (original_max - original_min) for row in cluster_dataX]
    cluster_center = np.mean(normalize_data, axis=0)
    cluster_centers.append(cluster_center)
cluster_centers = np.array(cluster_centers)

In [ ]:
# Load validation data
df = pd.read_excel("test_data_file/8-new validated instance_revison.xlsx")
validation_data = df.values
validation_dataX, validation_targetY = validation_data[:, :-1], np.log(validation_data[:, -1])

In [ ]:
# Preprocess validation data
data, target = (validation_dataX[7:8] - original_min) / (original_max - original_min), validation_targetY[7:8]

# Calculate distances to cluster centers
cluster_distance = [np.sqrt(np.sum((data - value) ** 2, axis=1)) for value in cluster_centers]
cluster_dis = [dis[0] for dis in cluster_distance]
cluster_index = np.argmin(cluster_distance)

In [ ]:
# Load model for the assigned cluster
model_name = "cluster_" + str(cluster_index + 1) + "SVR.model"
clf = joblib.load("model_for_custers/" + model_name)

# Predict creep rupture life
predicted_life = np.exp(clf.predict(data))[0]

In [ ]:
# Print results
print("True creep rupture life:", np.exp(target)[0])
print("Predicted creep rupture life:", predicted_life)